# Análisis y Segmentación De Clientes Ecommerce

## Instrucciones

- Este libro contiene el código fuente de los algoritmos utilizados para llegar el resultado deseado.
- Contiene gráficos descriptivos para ver como se comportan los datos y resultados de los mismos.

### Importando las librerías que se van a utilizar.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Exploración de los datos

In [2]:
# Carga y lectura del dataset
df = pd.read_csv('../Datasets/online_ecommerce.csv',sep=';')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
# Sumario estadístico
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [4]:
# Veamos las propiedades de las características
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [9]:
# Vamos a corregir el feature InvoiceDate
df['InvoiceDate'] = df['InvoiceDate'].astype('datetime64[ns]')

In [10]:
# Veamos si invoiceDate esta corregido
df['InvoiceDate'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 541909 entries, 0 to 541908
Series name: InvoiceDate
Non-Null Count   Dtype         
--------------   -----         
541909 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 4.1 MB


In [5]:
# Ver la dimensionalidad de los datos
df.shape

(541909, 8)

#### Este dataframe contiene 8 variables:

| Variable    | N ° Atributos | Descripción                                                                                                                                             |
|-------------|---------------|---------------------------------------------------------------------------------------------------------------------------------------------------------|
| InvoiceNo   | 25900         | Número de factura. Nominal, un número de 6 dígitos asignado de forma exclusiva a cada transacción. Si este código empieza en "c", indica una anulación. |
| StockCode   | 4070          | Código del producto (artículo). Nominal, un número de 5 dígitos asignado de forma exclusiva a cada producto distinto.                                   |
| Description | 4224          | Descripción (artículo). Nominal, descripción del producto en general.                                                                                   |
| Quantity    | 722           | Las cantidades de cada producto (artículo) por transacción. Numérico.                                                                                   |
| InvoiceDate | 23260         | Fecha y hora de la factura. Numérico, el día y la hora en que se generó cada transacción.                                                               |
| UnitPrice   | 1630          | Precio por unidad. Numérico, Precio del producto por unidad.                                                                                            |
| CustomerID  | 4373          | Número de cliente. Nominal, un número  de 5 dígitos asignado de forma exclusiva a cada cliente.                                                         |
| Country     | 38            | Nombre del país. Nominal, el nombre del país donde reside cada cliente.                                                                                 |


In [7]:
# Traemos la cantidad de atributos existentes en las features, por lo que cuadra con la tabla anterior
arr_col = df.columns.values
print("==== N° De Atributos ====")
for columna in arr_col:
    print(f"{columna}=>",len(df[columna].unique()))
print("=========================")

==== N° De Atributos ====
InvoiceNo=> 25900
StockCode=> 4070
Description=> 4224
Quantity=> 722
InvoiceDate=> 23260
UnitPrice=> 1630
CustomerID=> 4373
Country=> 38


In [12]:
columnas = pd.DataFrame(df.dtypes).T.rename(index={0: 'Tipo Columnas'})
total_re = pd.DataFrame(df.count()).T.rename(index={0:'Total Registros'})
valores_n = pd.DataFrame(df.isnull().sum()).T.rename(index={0:'Valores Nulos (Cantidad)'})
valores_p = pd.DataFrame((df.isnull().sum()/df.shape[0])* 100).T.rename(index={0:'Valores Nulos (%)'})
table_info = pd.concat([columnas,total_re,valores_n,valores_p],axis=0)
table_info

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
Tipo Columnas,object,object,object,int64,datetime64[ns],float64,float64,object
Total Registros,541909,541909,540455,541909,541909,541909,406829,541909
Valores Nulos (Cantidad),0,0,1454,0,0,0,135080,0
Valores Nulos (%),0.0,0.0,0.268311,0.0,0.0,0.0,24.926694,0.0


In [14]:
# Veamos el periodo de nuetro dataset
print(f"El dataset comienza con el punto de tiempo {df.InvoiceDate.min()}")
print(f"El dataset finaliza con el punto de tiempo {df.InvoiceDate.max()}")

El dataset comienza con el punto de tiempo 2010-12-01 08:26:00
El dataset finaliza con el punto de tiempo 2011-12-09 12:50:00


In [17]:
# Podemos Ver que el 1,71% corresponde a productos cancelados, por lo cuál podemos tratar estos datos a futuro
prod_can = pd.DataFrame()
prod_can['IsCancelled']=np.where(df.InvoiceNo.apply(lambda l: l[0] == 'C'),True,False)
prod_can.IsCancelled.value_counts() / prod_can.shape[0] * 100

False    98.286059
True      1.713941
Name: IsCancelled, dtype: float64

In [20]:
# Podemos encontrar en el total del dataset el 0.15% de productos en descuentos
prod_dis = pd.DataFrame()
prod_dis['IsDiscount'] = np.where(df.StockCode.apply(lambda d: d[0] == 'D'),True,False)
prod_dis.IsDiscount.value_counts() / prod_dis.shape[0] * 100

False    99.846469
True      0.153531
Name: IsDiscount, dtype: float64

In [26]:
#  Tenemos 5268 registros duplicados, pero hay que verificar si realmente son replicas
df.duplicated().sum()

5268

In [27]:
# Verificamos los duplicados, pero como tienen numero de factura, no significa que todos sean el mismo item
df[df.InvoiceNo.duplicated()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [59]:
print(f"""Cantidad Descuentos: {len(df[df['StockCode'] == 'D'])},
Cantidad envio por correo: {len(df[df['StockCode'] == 'POST'])},
Orden Canceladas: {len(df[df['InvoiceNo'].str.startswith('C')])}
""")

Cantidad Descuentos: 77,
Cantidad envio por correo: 1256,
Orden Canceladas: 9288

